In [125]:
import geopandas as gpd
from shapely import wkt
import pandas as pd
import numpy as np
import shapely

In [126]:
flows = pd.read_csv("./good_data/hhead_flows_wcoords_v4.csv")
flows.head()

,crsyr1,origin_loc,cross_loc,us_loc,origin_coords,cross_coords,us_coords
0,1972,Guanajuato,"Tijuana, Baja California del Norte","Chicago, IL","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-87.6244212, 41.8755616)"
1,1980,Guanajuato,"Tijuana, Baja California del Norte","San Diego, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-117.1627728, 32.7174202)"
2,1978,Guanajuato,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-118.2400932, 33.7689643)"
3,1980,Guanajuato,"Tijuana, Baja California del Norte","San Jose, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-121.890583, 37.3361905)"
4,1975,Guanajuato,"Nuevo Laredo, Tamaulipas","Chicago, IL","(-101.0, 20.9876996)","(-99.5185992, 27.4761506)","(-87.6244212, 41.8755616)"


In [127]:
flow_lines = []
for col, row in flows.iterrows():

    try:
    
        origin_coords = row.origin_coords
        cross_coords = row.cross_coords
        us_coords = row.us_coords

        origin_lat = float(origin_coords.split(", ")[0].replace("(", ""))
        origin_lon = float(origin_coords.split(", ")[1].replace(")", ""))

        cross_lat = float(cross_coords.split(", ")[0].replace("(", ""))
        cross_lon = float(cross_coords.split(", ")[1].replace(")", ""))

        us_lat = float(us_coords.split(", ")[0].replace("(", ""))
        us_lon = float(us_coords.split(", ")[1].replace(")", ""))    

        flow_lines.append(shapely.geometry.LineString([(origin_lat, origin_lon), (cross_lat, cross_lon), (us_lat, us_lon)]))
        
    except:
        
        flow_lines.append(np.nan)
        
flows['flow'] = flow_lines
flows.head()

,crsyr1,origin_loc,cross_loc,us_loc,origin_coords,cross_coords,us_coords,flow
0,1972,Guanajuato,"Tijuana, Baja California del Norte","Chicago, IL","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-87.6244212, 41.8755616)","LINESTRING (-101 20.9876996, -116.964662 32.50..."
1,1980,Guanajuato,"Tijuana, Baja California del Norte","San Diego, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-117.1627728, 32.7174202)","LINESTRING (-101 20.9876996, -116.964662 32.50..."
2,1978,Guanajuato,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-118.2400932, 33.7689643)","LINESTRING (-101 20.9876996, -116.964662 32.50..."
3,1980,Guanajuato,"Tijuana, Baja California del Norte","San Jose, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-121.890583, 37.3361905)","LINESTRING (-101 20.9876996, -116.964662 32.50..."
4,1975,Guanajuato,"Nuevo Laredo, Tamaulipas","Chicago, IL","(-101.0, 20.9876996)","(-99.5185992, 27.4761506)","(-87.6244212, 41.8755616)","LINESTRING (-101 20.9876996, -99.5185992 27.47..."


In [129]:
# flows["origin_coords"] = "POINT" + flows["origin_coords"].str.replace(", ", " ")
flows = flows[flows["crsyr1"] != 9999]
flows = flows.dropna(subset = ['flow'])
# flows['flow'] = flows['flow'].apply(wkt.loads)
flows = gpd.GeoDataFrame(flows, geometry = 'flow')
print(flows.shape)
flows.head()

(5052, 8)


,crsyr1,origin_loc,cross_loc,us_loc,origin_coords,cross_coords,us_coords,flow
0,1972,Guanajuato,"Tijuana, Baja California del Norte","Chicago, IL","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-87.6244212, 41.8755616)","LINESTRING (-101.00000 20.98770, -116.96466 32..."
1,1980,Guanajuato,"Tijuana, Baja California del Norte","San Diego, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-117.1627728, 32.7174202)","LINESTRING (-101.00000 20.98770, -116.96466 32..."
2,1978,Guanajuato,"Tijuana, Baja California del Norte","Los Angeles-Long Beach, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-118.2400932, 33.7689643)","LINESTRING (-101.00000 20.98770, -116.96466 32..."
3,1980,Guanajuato,"Tijuana, Baja California del Norte","San Jose, CA","(-101.0, 20.9876996)","(-116.964662, 32.5010188)","(-121.890583, 37.3361905)","LINESTRING (-101.00000 20.98770, -116.96466 32..."
4,1975,Guanajuato,"Nuevo Laredo, Tamaulipas","Chicago, IL","(-101.0, 20.9876996)","(-99.5185992, 27.4761506)","(-87.6244212, 41.8755616)","LINESTRING (-101.00000 20.98770, -99.51860 27...."


In [131]:
flows.to_file('./flow_shp/flow_shp.shp')

In [132]:
# flows["cross_lat"] = flows['cross_coords'].str.split(", ").str[0].str.replace("(", "")
# flows["cross_lon"] = flows['cross_coords'].str.split(", ").str[1].str.replace(")", "")

# flows["origin_lat"] = flows['origin_coords'].str.split(", ").str[0].str.replace("(", "")
# flows["origin_lon"] = flows['origin_coords'].str.split(", ").str[1].str.replace(")", "")

# flows["us_lat"] = flows['us_coords'].str.split(", ").str[0].str.replace("(", "")
# flows["us_lon"] = flows['us_coords'].str.split(", ").str[1].str.replace(")", "")

# # flows["cross_loc_coords"] = "POINT" + flows["cross_loc_coords"]
# # flows
# cross_locs = []
# for col, row in flows.iterrows():
# #     print(row)
#     try:
#         cross_locs.append(shapely.geometry.Point(float(row.cross_lat), float(row.cross_lon)))
#     except:
#         cross_locs.append(-99)
# flows['cross_loc_coords'] = cross_locs
      
    
# us_locs = []
# for col, row in flows.iterrows():
# #     print(row)
#     try:
#         us_locs.append(shapely.geometry.Point(float(row.us_lat), float(row.us_lon)))
#     except:
#         us_locs.append(-99)
# flows['us_coords'] = us_locs

        
        
# origin_locs = []
# for col, row in flows.iterrows():
# #     print(row)
#     try:
#         origin_locs.append(shapely.geometry.Point(float(row.origin_lat), float(row.origin_lon)))
#     except:
#         origin_locs.append(-99)
# flows['origin_coords'] = origin_locs

# flows.head()